In [1]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt') # one time execution
import re
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [4]:
df= pd.read_csv("FFXIV.csv",encoding = "ISO-8859-1")

In [5]:
df.head()

,ARTICLE,ARTICLE_TEXT,SOURCE
0,1,"Welcome to FINAL FANTASY? XIV (the ""Game"" or t...",https://support.na.square-enix.com/rule.php?id...
1,2,LICENSE LIMITATIONS\nThe Game is a carefully c...,https://support.na.square-enix.com/rule.php?id...
2,3,PROHIBITED ACTIVITIES\nIn addition to the Lice...,https://support.na.square-enix.com/rule.php?id...
3,4,BILLING PROCEDURES AND RECURRING TRANSACTIONS...,https://support.na.square-enix.com/rule.php?id...
4,5,INTERRUPTION TO THE SERVICE\nFrom time to time...,https://support.na.square-enix.com/rule.php?id...


In [8]:
df['ARTICLE_TEXT'][0]

'Welcome to FINAL FANTASY? XIV (the "Game" or the "Service"). In order to play the Game, you ("You" or "User") must first agree to this FINAL FANTASY XIV User Agreement ("User Agreement") between you and Square Enix, Inc. ("Square Enix"). PLEASE READ THIS USER AGREEMENT CAREFULLY. If you agree by checking the box next to the statement "I accept the User Agreement" or "I accept the terms of the above agreement(s)" you declare that you have read this User Agreement, you understand the terms of this User Agreement, and you agree to be bound by this User Agreement on behalf of yourself and, if you are under the age of 18, on your behalf by your parent or legal guardian. Once you agree, you will be contractually bound to the terms below, which shall govern all aspects of your use of the Game. No one under the age of 13 may play the Game.If you reject this User Agreement, then you will not be authorized to play the Game. In that case, you may either return the Game to the place of purchase s

separate text in sentences

In [9]:
sentences=[]

for s in df['ARTICLE_TEXT']:
    sentences.append(sent_tokenize(s))
    
sentences = [y for x in sentences for y in x] # flatten list

In [10]:
sentences[:5]

['Welcome to FINAL FANTASY?',
 'XIV (the "Game" or the "Service").',
 'In order to play the Game, you ("You" or "User") must first agree to this FINAL FANTASY XIV User Agreement ("User Agreement") between you and Square Enix, Inc. ("Square Enix").',
 'PLEASE READ THIS USER AGREEMENT CAREFULLY.',
 'If you agree by checking the box next to the statement "I accept the User Agreement" or "I accept the terms of the above agreement(s)" you declare that you have read this User Agreement, you understand the terms of this User Agreement, and you agree to be bound by this User Agreement on behalf of yourself and, if you are under the age of 18, on your behalf by your parent or legal guardian.']

Extract word vectors

In [11]:
word_embeddings = {}
f = open('D:/glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [12]:
len(word_embeddings)

400000

In [13]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lsosa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words_esp=stopwords.words('spanish')

# function to remove stopwords english
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# function to remove stopwords spanish
def remove_stopwords_esp(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words_esp])
    return sen_new


In [16]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences = [remove_stopwords_esp(r.split()) for r in clean_sentences]

In [17]:
# Extract word vectors
word_embeddings = {}
f = open('D:/glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [18]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [19]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [20]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [21]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph,alpha=0.85, personalization=None, max_iter=200)

#scores = nx.pagerank(nx_graph)

In [22]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [25]:
# Extract top 10 sentences as the summary
for i in range(15):
  print(ranked_sentences[i][1])

You agree to indemnify and hold harmless Square Enix and its parent, subsidiaries, affiliates, partners, officers, employees and agents from any claim, demand, or loss (including reasonable attorney's fees and court costs) incurred due to your usage of the Game, your engagement in Prohibited Activities, and/or arising out of or related to content you submit, post, link to, transmit, or make available through the Game, your violation of this User Agreement, or your violation of any rights of another User.
In its discretion, Square Enix may use a variety of methods to deal with violations of this User Agreement and the Prohibited Activities, including, but not limited to, any of the following:

Issuing a warning;
Placing a character in a "virtual jail" for a specified period of time;
Removing or deleting ill-gotten in-game items or currency;
Temporarily suspending an account;
Permanently terminating an account;
Permanently banning your IP address, residential address, or credit card numb